In [50]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import pandas as pd
from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Dropout, Lambda, Flatten, BatchNormalization
from keras.layers import Conv2D, MaxPool2D, AvgPool2D

# optimizer, data generator and learning rate reductor
from keras.optimizers import Adam

--------

**USING ONLY DENSE LAYERS**

--------

In [8]:
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):                   #on_epoch_end is called when an epoch ends, if the name of function is changed then it won't work
        print("log: " + str(logs))
        if(logs.get('loss')<0.2):
            print("\nReached 60% accuracy so cancelling training!")
            self.model.stop_training = True

callbacks = myCallback()

def load_data():
    fashion_mnist = keras.datasets.fashion_mnist
    return fashion_mnist.load_data()
    
def create_model():
    model = keras.Sequential([
    keras.layers.Flatten(),
    keras.layers.Dense(256,activation=tf.nn.relu),
    #keras.layers.Dense(128,activation=tf.nn.relu),
    keras.layers.Dense(10, activation=tf.nn.softmax)
])
    model.compile(optimizer=tf.train.AdamOptimizer(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

def train_and_evaluate__model(model, trainx, trainy, testx, testy):
    model.fit(trainx, trainy,epochs=3, callbacks = [callbacks])
    print(model.evaluate(testx, testy))
    
(trainx, trainy), (testx, testy) = load_data()
trainx = np.array(trainx)/255.0
testx = np.array(testx)/255.0
model = None
model = create_model()
train_and_evaluate__model(model, trainx, trainy, testx, testy)

Epoch 1/3
60000/60000 [==============================] - 10s 160us/step - loss: 0.4836 - acc: 0.8269
log: {'loss': 0.4835913896481196, 'acc': 0.82695}
Epoch 2/3
60000/60000 [==============================] - 9s 148us/step - loss: 0.3648 - acc: 0.8669
log: {'loss': 0.36480052959521614, 'acc': 0.86695}
Epoch 3/3
60000/60000 [==============================] - 9s 149us/step - loss: 0.3256 - acc: 0.8796
log: {'loss': 0.3255963470419248, 'acc': 0.8796}
10000/10000 [==============================] - 0s 38us/step
[0.3698438768386841, 0.8671]


In [9]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_5 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_10 (Dense)             (None, 256)               200960    
_________________________________________________________________
dense_11 (Dense)             (None, 10)                2570      
Total params: 203,530
Trainable params: 203,530
Non-trainable params: 0
_________________________________________________________________


--------

**ADDING 1 CONVOLUTION LAYER AND 1 MAXPOOLING LAYER**

--------

In [72]:
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):                   #on_epoch_end is called when an epoch ends, if the name of function is changed then it won't work
        print("log: " + str(logs))
        if(logs.get('loss')<0.0001):
            #print("\nReached 80% accuracy so cancelling training!")
            self.model.stop_training = True

callbacks = myCallback()


train = pd.read_csv("/Users/vaibhav/Downloads/train.csv")
trainy = train['label']
trainx = train.drop("label", axis=1)

testx = pd.read_csv("/Users/vaibhav/Downloads/test.csv")
    
def create_model():
    model = Sequential()

    model.add(Conv2D(64, kernel_size = 3, activation='relu', input_shape = (28,28,1)))
    model.add(BatchNormalization())
    model.add(Conv2D(64, kernel_size = 3, activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(64, kernel_size = 5, strides=2, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))

    model.add(Conv2D(128, kernel_size = 3, activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(128, kernel_size = 3, activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(128, kernel_size = 5, strides=2, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))

    model.add(Conv2D(256, kernel_size = 4, activation='relu'))
    model.add(BatchNormalization())
    model.add(Flatten())
    model.add(Dropout(0.4))
    model.add(Dense(10, activation='softmax'))
    
    model.compile(optimizer=tf.optimizers.Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

def train_and_evaluate__model(model, trainX, trainY, testX):
    model.fit(trainX, trainY,epochs=20, callbacks = [callbacks], batch_size=64)
    #print(model.evaluate(x=testX, y=testY))
    predictions = model.predict(testX)
    

trainx = trainx.values.reshape(42000,28,28,1)           #for convolution layer(it expects everything in single tensor[4-d] and not 60000 images as a list in dense layer)
testx = testx.values.reshape(28000,28,28,1)             #for convolution layer


#one hot encoder label
trainy = to_categorical(trainy, num_classes = 10)

#trainX, testX, trainY, testY = train_test_split(trainx, trainy, test_size = 0.1)

#normalisation
trainx = np.array(trainx)/255.0
testx = np.array(testx)/255.0
model = None
model = create_model()
train_and_evaluate__model(model, trainx, trainy, testx)

Epoch 1/20
42000/42000 [==============================] - 234s 6ms/step - loss: 0.1760 - accuracy: 0.9468
log: {'loss': 0.1759790851147402, 'accuracy': 0.9467857}
Epoch 2/20
42000/42000 [==============================] - 242s 6ms/step - loss: 0.0587 - accuracy: 0.9818
log: {'loss': 0.058706330619220225, 'accuracy': 0.98183334}
Epoch 3/20
42000/42000 [==============================] - 251s 6ms/step - loss: 0.0460 - accuracy: 0.9859
log: {'loss': 0.0460444989808436, 'accuracy': 0.98590475}
Epoch 4/20
42000/42000 [==============================] - 261s 6ms/step - loss: 0.0388 - accuracy: 0.9875
log: {'loss': 0.03883004041539417, 'accuracy': 0.98754764}
Epoch 5/20
42000/42000 [==============================] - 270s 6ms/step - loss: 0.0355 - accuracy: 0.9885
log: {'loss': 0.03546970651156846, 'accuracy': 0.9885}
Epoch 6/20
42000/42000 [==============================] - 274s 7ms/step - loss: 0.0313 - accuracy: 0.9900
log: {'loss': 0.03128993932082362, 'accuracy': 0.9900238}
Epoch 7/20
42000/

In [73]:
predictions = model.predict(testx)

In [74]:
predictions = np.argmax(predictions,axis = 1)

In [75]:
ImageId = []
for i in range(1, 28001):
    ImageId.append(i)
df = pd.DataFrame(columns = {"ImageId"}, data = ImageId)
df["Label"] = predictions
df.to_csv("/Users/vaibhav/Downloads/predictions.csv", index=False)